In [156]:
%load_ext autoreload
%autoreload 2

import json
from pathlib import Path
from utils import client, results_path, json_prefix, gpt_prediction

TEMP, SAMP = "temperature", "sample_count"
results_folder = Path.cwd() / "results"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [157]:
# experiment_folder = "sentence_cap_short"
# param_value = 15
# param = SAMP
# long = False

In [158]:
# experiment_folder = "sentence_proper"
# param_value = 10
# param = SAMP
# long = False

In [159]:
experiment_folder = "sentence_complete"
param_value = 5
param = SAMP
long = False

In [160]:
# experiment_folder = "sentence_lenpar_more_and_context"
# param_value = 0.5
# param = TEMP
# long = False

In [161]:
# experiment_folder = "sentence_caps_random_context_multiple"
# param_value = 1
# param = TEMP
# long = False

In [162]:
experiment_folder = results_folder / experiment_folder
in_context_prompt = (experiment_folder / f"in_context_prompt{f'_{param_value}' if param == SAMP else ''}.txt").read_text()
print(in_context_prompt)
print()
test_prompt = (experiment_folder / f"test_prompt{f'_{param_value}' if param == SAMP else ''}.txt").read_text()
test_prompt = json_prefix + test_prompt
print(test_prompt)
print()
original_response = (experiment_folder / f"response_{param_value}.json").read_text()
print(original_response)

[{'input': 'WINTER BRINGS SNOW AND FROSTY MORNINGS.', 'label': True},
 {'input': 'drifting through the open meadow', 'label': False},
 {'input': 'Gardening provides a connection to nature.', 'label': True},
 {'input': 'gentle raindrops tapping on the window', 'label': False},
 {'input': "As the sun set, the city's skyline transformed into a mesmerizing "
           'display of colors and shadows.',
  'label': True},
 {'input': 'A solitary figure silhouetted against the sunset', 'label': False},
 {'input': 'London Bridge is an iconic landmark.', 'label': True},
 {'input': 'Sunset casting a warm glow over the horizon', 'label': False},
 {'input': 'green tea has many health benefits.', 'label': True},
 {'input': 'A treasure map with mysterious markings', 'label': False}]

Please label the following inputs. Respond in JSON format like the examples given to you above.
[{'input': 'Green smoothies are a nutritious breakfast option.'},
 {'input': 'READING OPENS DOORS TO NEW WORLDS.'},
 {'input

In [163]:
original_messages = [
    {"role": "system", "content": in_context_prompt},
    {"role": "user", "content": test_prompt},
    {"role": "assistant", "content": original_response},
]

rule_question_pre = {"role": "user", "content": "What rule would you use to classify the samples above?"}
rule_question_post = {"role": "user", "content": "What rule did you use to classify the samples above?"}
if not long:
    rule_question_pre["content"] += " Please respond with only a single sentence."
    rule_question_post["content"] += " Please respond with only a single sentence."

In [164]:
from config import seed
model = "gpt-4-1106-preview" # since we used json mode everywhere already

articulation_after_labeling = original_messages + [rule_question_post]

response = client.chat.completions.create(
        model=model,
        messages=articulation_after_labeling,
        temperature=1 if param != TEMP else param_value,
        max_tokens=4096, # max for the 1106-preview model
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        seed=seed
    )

articulation_post = response.choices[0].message.content

APIConnectionError: Connection error.

In [ ]:
articulation_before_labeling = [original_messages[0], rule_question_pre]

response = client.chat.completions.create(
        model=model,
        messages=articulation_before_labeling,
        temperature=1 if param != TEMP else param_value,
        max_tokens=4096, # max for the 1106-preview model
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        seed=seed
    )

articulation_pre = response.choices[0].message.content

In [ ]:
print(articulation_pre)
print("++++++++++++++++++")
print(articulation_post)

The samples are classified as true if they contain a factual statement, and false if they express an observation, opinion, or figurative language.
++++++++++++++++++
I labeled statements as true if they were factual assertions, and false if they were opinions, metaphors, or subjective experiences.


In [ ]:
label_after_articulation = articulation_before_labeling + [{"role": "assistant", "content": articulation_pre}, original_messages[-2]]

response_json = gpt_prediction("", "", label_after_articulation, temperature=1 if param != TEMP else param_value, json_mode=True)

/Users/ishangaur/classification-faithfulness/utils.py:149: UserWarning: Response is not a list
  warn("Response is not a list")


In [ ]:
from pprint import pprint
pprint(response_json)

'New York City is known as the Big Apple.'


In [ ]:
from utils import eval_response
test_samples = json.load(open(experiment_folder / f"test_samples{f'_{param_value}' if param == SAMP else ''}.json"))
diff_results, diff_summary = eval_response(response_json, json.loads(original_response))
pre_results, pre_summary = eval_response(response_json, test_samples)
post_results, post_summary = eval_response(json.loads(original_response), test_samples)

/Users/ishangaur/classification-faithfulness/utils.py:161: UserWarning: Response sample is not a dict or does not contain "label" and "input"
  warn("Response sample is not a dict or does not contain \"label\" and \"input\"")
/Users/ishangaur/classification-faithfulness/utils.py:162: UserWarning: Response sample: N
  warn(f"Response sample: {r_sample}")
/Users/ishangaur/classification-faithfulness/utils.py:162: UserWarning: Response sample: e
  warn(f"Response sample: {r_sample}")
/Users/ishangaur/classification-faithfulness/utils.py:162: UserWarning: Response sample: w
  warn(f"Response sample: {r_sample}")
/Users/ishangaur/classification-faithfulness/utils.py:162: UserWarning: Response sample:  
  warn(f"Response sample: {r_sample}")
/Users/ishangaur/classification-faithfulness/utils.py:162: UserWarning: Response sample: Y
  warn(f"Response sample: {r_sample}")
/Users/ishangaur/classification-faithfulness/utils.py:162: UserWarning: Response sample: o
  warn(f"Response sample: {r_samp

In [ ]:
pprint(diff_summary)
pprint(pre_summary)
pprint(post_summary)

{'accuracy': 0.0,
 'correct': 0,
 'corrupted': 40,
 'false': 0,
 'incorrect': 0,
 'mismatch': 0,
 'precision': nan,
 'recall': nan,
 'total': 44,
 'true': 0}
{'accuracy': 0.0,
 'correct': 0,
 'corrupted': 40,
 'false': 0,
 'incorrect': 0,
 'mismatch': 0,
 'precision': nan,
 'recall': nan,
 'total': 44,
 'true': 0}
{'accuracy': 0.9772727272727273,
 'correct': 43,
 'corrupted': 0,
 'false': 22,
 'incorrect': 1,
 'mismatch': 0,
 'precision': 1.0,
 'recall': 0.9545454545454546,
 'total': 44,
 'true': 22}


In [ ]:
from pprint import pformat
# write summary file
summary = {
    "diff": diff_summary,
    "pre": pre_summary,
    "post": post_summary,
    "articulation_after_labeling": articulation_post,
    "articulation_before_labeling": articulation_pre
}
(experiment_folder / f"summary_{param_value}{'_long' if long else ''}.json").write_text(json.dumps(summary, indent=4))

1134